In [1]:
#Importacion de las librerias
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from datetime import datetime
import psycopg2
import pyodbc

In [62]:
#Creacion de los data frame de la base de datos chinook
engine = create_engine('sqlite:///Northwind_Larg.sqlite')
df_customer = pd.read_sql_query("""SELECT Id as Id_Customer,CompanyName, ContactName, ContactTitle, Address, City,Region,PostalCode,Country,Phone,Fax FROM Customer; """, con=engine.connect())
df_location = pd.read_sql_query("""SELECT ShipVia,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry FROM Orders; """, con=engine.connect())
df_employees = pd.read_sql_query("""SELECT LastName,FirstName,Title,TitleOfCourtesy,BirthDate,HireDate,Address,City,Region,PostalCode,Country,Extension,Notes,PhotoPath,t.TerritoryDescription,r.RegionDescription FROM Employee e
    INNER JOIN
    EmployeeTerritory et  
    INNER JOIN
    Territory t ON t.Id = et.TerritoryId
    INNER JOIN
    Region r ON r.Id = t.RegionId    
    ; """, con=engine.connect())
df_products = pd.read_sql_query("""SELECT ProductName,CategoryName,Description,QuantityPerUnit as QuanntityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued as Discount FROM Product p
    INNER JOIN
    Category c ON c.Id = p.CategoryId
    ; """, con=engine.connect())
df_suppliers = pd.read_sql_query("""SELECT CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax FROM Customer; """, con=engine.connect())
 
df_so = pd.read_sql_query("""SELECT OrderDetail.UnitPrice,OrderDetail.Quantity,OrderDetail.Discount,Orders.Freight,Customer.Id as Id_Customer,Employee.Id as Id_Employee,Orders.Id as Id_Location,Supplier.Id as Id_Supplier,Product.Id as Id_Product,TimeId as Id_Time 
    FROM Orders
    INNER JOIN
    OrderDetail ON OrderDetail.OrderId = Orders.Id
    INNER JOIN
    Customer ON Customer.Id = Orders.CustomerId
    INNER JOIN
    Employee ON Employee.Id = Orders.EmployeeId
    INNER JOIN
    Product ON Product.Id = OrderDetail.ProductId
    INNER JOIN
    Supplier ON Supplier.Id = Product.SupplierId 
    INNER JOIN 
    Dim_Time ON Orders.OrderDate= substr(Dim_Time.date,0,11)
    ; """, con=engine.connect())

In [55]:
#Creacion del data frame de fecha
fecha_I='2009-01-01'
fecha_F='2030-12-31'
df_date = pd.DataFrame({'date': pd.date_range(fecha_I, fecha_F)})
df_date['año'] = df_date.date.dt.year
df_date['mes'] = df_date.date.dt.month
df_date['dia'] = df_date.date.dt.day
df_date['nombre_dia'] = df_date.date.dt.day_name()
df_date['dia_semana'] = df_date.date.dt.dayofweek
df_date['semana'] = df_date.date.dt.weekofyear
df_date['trimestre'] = df_date.date.dt.quarter
df_date = df_date[['date', 'año', 'mes', 'dia', 'nombre_dia', 'dia_semana', 'semana', 'trimestre']]
df_date.to_sql(name='Dim_Time', con=engine.connect(), if_exists='append',index=False)

C:\Users\pined\AppData\Local\Temp/ipykernel_20572/1429112024.py:10: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df_date['semana'] = df_date.date.dt.weekofyear


8035

In [59]:
#Conexion con SQL
conn = sqlalchemy.create_engine('mssql+pyodbc://localhost/Estrella?driver=SQL+Server+Native+Client+11.0')

In [63]:
#Limpieza de los datos
df_location.fillna(method="bfill", inplace = True)
df_employees.fillna(method="bfill", inplace = True)
df_customer.fillna(method="bfill", inplace = True)
df_products.fillna(method="bfill", inplace = True)
df_suppliers.fillna(method="bfill", inplace = True)
df_location.fillna(method="ffill", inplace = True)
df_employees.fillna(method="ffill", inplace = True)
df_customer.fillna(method="ffill", inplace = True)
df_products.fillna(method="ffill", inplace = True)
df_suppliers.fillna(method="ffill", inplace = True)

In [7]:
#Carga de los datos en SQL
df_customer.to_sql(name='Dim_Customer', con=conn, if_exists='append',index=False)
df_location.to_sql(name='Dim_Location', con=conn, if_exists='append',index=False)
df_employees.to_sql(name='Dim_Employees', con=conn, if_exists='append',index=False)
df_products.to_sql(name='Dim_Products', con=conn, if_exists='append',index=False)
df_suppliers.to_sql(name='Dim_Suppliers', con=conn, if_exists='append',index=False)
df_date.to_sql(name='Dim_Time', con=conn, if_exists='append',index=False)
df_so.to_sql(name='Sales_Order', con=conn, if_exists='append',index=False)

-1